In [ ]:
%pip install wandb
%pip install datasets 

In [3]:
import random 
import torch   
import torch.nn as nn 
import torch.nn.functional as F
import math  
from dataclasses import dataclass 
from tokenizers import Tokenizer 
from pathlib import Path 
from torch.utils.data import Dataset , DataLoader 
import wandb 
from transformers import AutoTokenizer , AutoModelForCausalLM

In [8]:
@dataclass 
class Config:
    dpo_model_name : str = "mistralai/Mistral-7B-Instruct",
    model_name : str = "Qwen/Qwen2-0.5B-Instruct",
    dataset_name: str = "trl-lib/ultrafeedback_binarized",
    batch_size : int = 2 ,
    beta : float = 0.1 , 
    learning_rate : float = 1e-4 
    HF_TOKEN : str = "PASS"

In [ ]:
wandb.init(project="dpo-finetune" , 
           config = Config)


In [ ]:
class DPO:
    def __init__(self , config: Config):
        self.config = Config()
        self.ref_model = AutoModelForCausalLM.from_pretrained(self.config.dpo_model_name , token = self.config.HF_TOKEN)
        self.model = AutoModelForCausalLM.from_pretrained(self.config.model_name , token = self.config.HF_TOKEN) 
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name , token = self.config.HF_TOKEN) 
        self.ref_model.eval()
    
    def DPOLoss(self , datapoint):
        """Compute the DPO loss for a single datapoint"""
        
        
        